# 图像（数据）增广
- Data Augmentation

**核心问题：训练时的数据和实际使用时的数据长得不一样！**

### 图像增广就是提前模拟各种情况

```
你的训练数据：都是在办公室拍的，光线好，角度正

实际使用时：
    可能光线暗
    可能颜色偏黄
    可能只拍到物体的一半
    可能图片是翻转的

图像增广 = 把训练数据变换一下，模拟这些情况
    让模型提前见过各种"奇怪"的图片
    这样实际使用时就不会懵了
```
---

## 二、常用的增广方法（三种最重要）

### 方法1：翻转

```
左右翻转：最常用，几乎所有任务都用
    猫朝左 → 翻转 → 猫朝右
    一张图变两张图！

上下翻转：看情况用
    猫翻过来头朝下 → 很奇怪 → 一般不用
    树叶、卫星图 → 可以用（没有上下之分）
```

### 方法2：随机裁剪

```
从图片中随机切一块出来，再缩放到固定大小

为什么？
    实际拍照时，物体可能只占画面一部分
    可能只拍到猫的头
    可能只拍到猫的尾巴
    模型应该都能认出来

具体做法：
    随机选位置
    随机选大小（占原图8%~100%）
    随机选高宽比（3/4 到 4/3）
    切出来后缩放到224×224
```

### 方法3：颜色变换

```
随机改变亮度、色调、饱和度

为什么？
    不同灯光下颜色不同（白天vs晚上）
    不同相机拍出来颜色不同
    模型不应该因为颜色变了就认不出来

具体做法：
    亮度：在0.5到1.5之间随机（变暗或变亮）
    色调：随机偏红、偏蓝、偏黄
    饱和度：颜色浓一点或淡一点
```

---

## 三、什么时候用？

```
训练时：每次读图片都随机做增广
        同一张猫的图片，每次训练看到的都不一样
        → 相当于增加了数据量

测试/推理时：不做增广！用原始图片
        → 要稳定的结果，不能加随机性
```

**这也是一种正则化方法（和Dropout类似，只在训练时用）**

---

**要点与疑惑一一回答**
# 从根本问题开始：为什么需要Resize？



## 关键问题：网络能接受不同大小的图片吗？

### 答案：不能！（至少对于全连接层来说）

```python
# ResNet的最后是这样：
卷积层 → 全局池化 → Flatten → Linear(512, 10)
                              ^^^^^^^^
     这一层要求输入必须是512个数字

# 如果图片大小不同：
图片1（224×224）→ 卷积 → 池化 → Flatten → 512个数字 ✓
图片2（300×400）→ 卷积 → 池化 → Flatten → 738个数字 ✗
                                          ^^^
       大小变了！Linear层炸了！
```

**所以你必须让所有图片变成同样大小，才能进同一个网络。**

---

## 你可能会说：那我直接ToTensor不就行了吗？

### 看看直接ToTensor会发生什么

```python
# 只用ToTensor，不Resize
transform = transforms.ToTensor()

# 你的图片：
cat1.jpg → 原始大小 1200×800  → ToTensor → Tensor(3, 1200, 800)
cat2.jpg → 原始大小 300×400   → ToTensor → Tensor(3, 300, 400)
cat3.jpg → 原始大小 640×480   → ToTensor → Tensor(3, 640, 480)

# 做成DataLoader
train_loader = DataLoader(train_dataset, batch_size=64)

# 当你训练时
for X, y in train_loader:
    # DataLoader想把64张图打包成一个batch
    # 但64张图大小都不一样！
    # 打包失败！直接报错！
    
# 报错信息：
RuntimeError: stack expects each tensor to be equal size
```

**DataLoader要把多张图片打包成一个batch，它们必须一样大！**

---

## 所以必须Resize

```python
# 正确做法：先Resize再ToTensor
transform = transforms.Compose([
    transforms.Resize(224),        # 所有图片统一变成224×224
    transforms.ToTensor(),         # 再转成Tensor
])

# 现在：
cat1.jpg → 1200×800  → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)
cat2.jpg → 300×400   → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)
cat3.jpg → 640×480   → Resize → 224×224 → ToTensor → Tensor(3, 224, 224)

# 做成DataLoader
for X, y in train_loader:
    # 现在64张图都是224×224
    # 打包成功！X的形状是 (64, 3, 224, 224)
    # 可以丢进网络了
```

---

## 为什么需要transforms.Compose？

### 因为你要做两件事

```python
# 如果不用Compose，你要这样写：
图片 = 读取图片()
图片 = Resize(图片)        # 手动做第1步
图片 = ToTensor(图片)      # 手动做第2步

# 每读一张图都要写两行！太麻烦了！
```

```python
# 用Compose打包：
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

# 以后只要：
图片 = transform(原始图片)  # 自动执行两步
```

**Compose就是把多个操作打包，省得每次手动做。**

---

## 完整流程图

```
原始图片文件（cat.jpg，大小1200×800）
        │
        │ 不Resize直接ToTensor会怎样？
        ↓
    ToTensor
        │
        ↓
    Tensor(3, 1200, 800)  ← 每张图大小不一样
        │
        ↓
    DataLoader想打包成batch
        │
        ↓
    报错！无法打包不同大小的图片
```

```
原始图片文件（cat.jpg，大小1200×800）
        │
        │ 先Resize再ToTensor
        ↓
    Resize(224)
        │
        ↓
    PIL图片(224×224)  ← 现在所有图都一样大了
        │
        ↓
    ToTensor
        │
        ↓
    Tensor(3, 224, 224)  ← 所有Tensor大小都一样
        │
        ↓
    DataLoader打包成batch
        │
        ↓
    (64, 3, 224, 224)  ← 成功！可以训练
```

---

## 现在加上增广

### 没有增广（最简单）

```python
# 必须做的两件事
transform = transforms.Compose([
    transforms.Resize(224),        # 必须：统一大小
    transforms.ToTensor(),         # 必须：变成数字
])
```

### 有增广（训练时多做几步变换）

```python
# 训练时：在必须做的事之前加随机操作
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),     # 额外：随机翻转
    transforms.RandomResizedCrop(224),     # 代替Resize：随机裁剪到224
    transforms.ToTensor(),   # 必须：变成数字
])
```

### RandomResizedCrop = Resize的升级版

```
Resize(224)：
    每张图都缩放到224×224
    每次结果都一样

RandomResizedCrop(224)：
    随机切一块，再缩放到224×224
    每次结果都不一样（位置随机、大小随机）
    
效果：
    都能保证最后是224×224
    但RandomResizedCrop加了随机性（增广）
```

---

## 为什么测试时用Resize+CenterCrop而不是Resize？

```python
# 为什么不直接Resize(224)？
transforms.Resize(224)
# 问题：如果图片不是正方形，会被拉变形

原图：400×300（长方形）
    ┌────────┐
    │  🐱   │  ← 猫是正常比例
    └────────┘

直接Resize(224)后：224×224（正方形）
    ┌────┐
    │ 🐱 │  ← 猫被压扁了！
    └────┘
```

```python
# 更好的方法：先Resize(256)，再CenterCrop(224)
transforms.Resize(256)
# 保持比例缩放，短边变成256

原图：400×300
Resize(256)后：341×256（还是长方形，但小了）

transforms.CenterCrop(224)
# 从中间切出224×224

    ┌─────────┐
    │ ┌─────┐ │
    │ │ 🐱  │ │  ← 从中间切，猫不变形
    │ └─────┘ │
    └─────────┘
```

---

## 总结

```
为什么不能只ToTensor？
    → 不同大小的图片无法打包成batch
    → DataLoader直接报错

为什么需要Resize？
    → 让所有图片变成同样大小（比如224×224）
    → 这样才能打包成batch

为什么需要Compose？
    → 因为要做好几步操作（Resize + ToTensor）
    → Compose把它们打包成一个流程

增广是什么？
    → 在必须做的事（Resize/ToTensor）之前
    → 额外加几个随机操作（翻转、裁剪、变色）
    → 让模型多见世面

训练和测试的区别？
    train_transform：有随机操作（增广）
    test_transform：没有随机操作（要稳定结果）
    
    但最后都要ToTensor（这是必须的）
```

### 代码模板

```python
# 训练transform（有随机增广）
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, 
                           saturation=0.5, hue=0.1),
    transforms.ToTensor(),
])

# 测试transform（没有随机，固定操作）
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
```
